In [1]:
from lxml import etree
import pandas as pd
from io import StringIO

In [2]:
doc = etree.parse("fpds_feed_sample_1_org.xml")

In [3]:
xslt_tree = etree.parse("fpds_feed_v3.xsl")
transform = etree.XSLT(xslt_tree)

In [4]:
result = transform(doc)

In [5]:
csv_str =str(result)
csv_str

'ClingerCohenAct\tN\tN\tN\tN\tN\tN\tN\tN\tN\tY\nClingerCohenAct-description\tNO\tNO\tNO\tNO\tNO\tNO\tNO\tNO\tNO\tYES\nmaterialsSuppliesArticlesEquipment\tN\tN\tN\tN\tN\tN\tY\tN\tX\tX\nmaterialsSuppliesArticlesEquipment-description\tNO\tNO\tNO\tNO\tNO\tNO\tYES\tNO\tNOT APPLICABLE\tNOT APPLICABLE\nlaborStandards\tN\tN\tN\tN\tN\tN\tX\tY\tY\tY\nlaborStandards-description\tNO\tNO\tNO\tNO\tNO\tNO\tNOT APPLICABLE\tYES\tYES\tYES\nconstructionWageRateRequirements\tN\tN\tN\tN\tN\tN\tX\tN\tX\tY\nconstructionWageRateRequirements-description\tNO\tNO\tNO\tNO\tNO\tNO\tNOT APPLICABLE\tNO\tNOT APPLICABLE\tYES\ninteragencyContractingAuthority\tX\tX\tX\tX\tX\tX\tX\tX\tX\tX\ninteragencyContractingAuthority-description\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\tNOT APPLICABLE\notherStatutoryAuthority\t\t\t\t\t\t\t\t\t\t\nadditionalReportingValue\t\t\t\t\t\t\t\t\t\tNONE\n'

In [6]:
mystr = StringIO(csv_str)

df = pd.read_csv(mystr, sep="\t", header=None, index_col=0)
df.tail(16)

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
ClingerCohenAct,N,N,N,N,N,N,N,N,N,Y
ClingerCohenAct-description,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES
materialsSuppliesArticlesEquipment,N,N,N,N,N,N,Y,N,X,X
materialsSuppliesArticlesEquipment-description,NO,NO,NO,NO,NO,NO,YES,NO,NOT APPLICABLE,NOT APPLICABLE
laborStandards,N,N,N,N,N,N,X,Y,Y,Y
laborStandards-description,NO,NO,NO,NO,NO,NO,NOT APPLICABLE,YES,YES,YES
constructionWageRateRequirements,N,N,N,N,N,N,X,N,X,Y
constructionWageRateRequirements-description,NO,NO,NO,NO,NO,NO,NOT APPLICABLE,NO,NOT APPLICABLE,YES
interagencyContractingAuthority,X,X,X,X,X,X,X,X,X,X


In [7]:
df_new = df.transpose(copy=True)
df_new.iloc[:, -16:]

,ClingerCohenAct,ClingerCohenAct-description,materialsSuppliesArticlesEquipment,materialsSuppliesArticlesEquipment-description,laborStandards,laborStandards-description,constructionWageRateRequirements,constructionWageRateRequirements-description,interagencyContractingAuthority,interagencyContractingAuthority-description,otherStatutoryAuthority,additionalReportingValue
1,N,NO,N,NO,N,NO,N,NO,X,NOT APPLICABLE,NaN,NaN
2,N,NO,N,NO,N,NO,N,NO,X,NOT APPLICABLE,NaN,NaN
3,N,NO,N,NO,N,NO,N,NO,X,NOT APPLICABLE,NaN,NaN
4,N,NO,N,NO,N,NO,N,NO,X,NOT APPLICABLE,NaN,NaN
5,N,NO,N,NO,N,NO,N,NO,X,NOT APPLICABLE,NaN,NaN
6,N,NO,N,NO,N,NO,N,NO,X,NOT APPLICABLE,NaN,NaN
7,N,NO,Y,YES,X,NOT APPLICABLE,X,NOT APPLICABLE,X,NOT APPLICABLE,NaN,NaN
8,N,NO,N,NO,Y,YES,N,NO,X,NOT APPLICABLE,NaN,NaN
9,N,NO,X,NOT APPLICABLE,Y,YES,X,NOT APPLICABLE,X,NOT APPLICABLE,NaN,NaN
10,Y,YES,X,NOT APPLICABLE,Y,YES,Y,YES,X,NOT APPLICABLE,NaN,NONE


In [8]:
df_new.shape

(10, 12)

In [ ]:
# output dataframe to a .csv file
df_new.to_csv("fpds_feed_result.csv", index=False)